In [4]:
import math
import os
import sys
from pathlib import Path

import einops
import numpy as np
from jax import numpy as jnp

In [2]:
! wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip

--2025-03-17 17:53:04--  https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/callummcdougall/ARENA_3.0/zip/refs/heads/main [following]
--2025-03-17 17:53:05--  https://codeload.github.com/callummcdougall/ARENA_3.0/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [  <=>               ]  20.96M  1.81MB/s    in 14s     

2025-03-17 17:53:20 (1.50 MB/s) - ‘main.zip’ saved [21977808]



In [3]:
!unzip main.zip

Archive:  main.zip
a68ce8e24f44620cd21322cefeb84f2af62aeaca
   creating: ARENA_3.0-main/
  inflating: ARENA_3.0-main/.gitignore  
   creating: ARENA_3.0-main/.streamlit/
  inflating: ARENA_3.0-main/.streamlit/config.toml  
  inflating: ARENA_3.0-main/README.md  
   creating: ARENA_3.0-main/chapter0_fundamentals/
   creating: ARENA_3.0-main/chapter0_fundamentals/exercises/
   creating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_prereqs/
  inflating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_prereqs/0.0_Prerequisites_exercises.ipynb  
  inflating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_prereqs/0.0_Prerequisites_solutions.ipynb  
  inflating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_prereqs/numbers.npy  
  inflating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_prereqs/solutions.py  
  inflating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_prereqs/tests.py  
  inflating: ARENA_3.0-main/chapter0_fundamentals/exercises/part0_pr

In [5]:
import part0_prereqs.tests as tests
from part0_prereqs.utils import display_array_as_img, display_soln_array_as_img

In [35]:
arr = jnp.load("part0_prereqs/numbers.npy")

In [36]:
print(arr[0].shape)
display_array_as_img(arr[0])  # plotting the first image in the batch

(3, 150, 150)


In [37]:
print(arr[0, 0].shape)
display_array_as_img(arr[0, 0])  # plotting the first channel of the first image, as monochrome

(150, 150)


In [42]:
arr_stacked = einops.rearrange(arr, "b c h w -> c h (b w)")
print(arr_stacked.shape)
display_array_as_img(arr_stacked)

(3, 150, 900)


Einops

In [44]:
arr1 = einops.rearrange(arr, "b c h w -> c (b h) w")
print(arr1.shape)
display_array_as_img(arr1)

(3, 900, 150)


In [48]:
arr2 = einops.repeat(arr[0], "c h w -> c (2 h) w")
print(arr2.shape)
display_array_as_img(arr2)

(3, 300, 150)


In [58]:
arr3 = einops.repeat(arr[0:2], "b c h w -> c (b h) (2 w)")
print(arr3.shape)
display_array_as_img(arr3)

(3, 300, 300)


In [68]:
arr4 = einops.repeat(arr[0], "c h w -> c (h 2) w")
display_array_as_img(arr4)

In [73]:
arr5 = einops.rearrange(arr[0], "c h w -> h (c w)")
print(arr5.shape)
display_array_as_img(arr5)


(150, 450)


In [77]:
arr6 = einops.rearrange(arr, "(b1 b2) c h w -> c (b1 h) (b2 w)", b1=2)
print(arr6.shape)
display_array_as_img(arr6)

(3, 300, 450)


In [80]:
arr7 = einops.rearrange(arr[1], "c h w -> c w h")
print(arr7.shape)
display_array_as_img(arr7)

(3, 150, 150)


In [85]:
arr8 = einops.reduce(arr, "(b1 b2) c (h h2) (w w2) -> c (b1 h) (b2 w)", "max",  h2=2, w2=2, b1=2)
print(arr8.shape)
display_array_as_img(arr8)

(3, 150, 225)


Broadcasting

In [86]:
x = jnp.ones((3, 1, 5))
y = jnp.ones((1, 4, 5))

z = x + y

In [87]:
z

Array([[[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]],

       [[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]],

       [[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]]], dtype=float32)

In [101]:
def assert_all_equal(actual: jnp.array, expected: jnp.array) -> None:
    assert actual.shape == expected.shape, f"Shape mismatch, got: {actual.shape}"
    assert (actual == expected).all(), f"Value mismatch, got: {actual}"
    print("Tests passed!")

def assert_all_close(actual: jnp.array, expected: jnp.array, atol=1e-3) -> None:
    assert actual.shape == expected.shape, f"Shape mismatch, got: {actual.shape}"
    np.testing.assert_allclose(actual, expected, atol=atol, rtol=0.0)
    print("Tests passed!")

In [93]:
def rearrange_1() -> jnp.array:
    return einops.rearrange(jnp.arange(3, 9), "(h w) -> h w", h=3, w=2)
expected = jnp.array([[3, 4], [5, 6], [7, 8]])
assert_all_equal(rearrange_1(), expected)

Tests passed!


In [97]:
def rearrange_2() -> jnp.array:
    return einops.rearrange(jnp.arange(1, 7), "(h w) -> h w", h=2, w=3)

assert_all_equal(rearrange_2(), jnp.array([[1, 2, 3], [4, 5, 6]]))

Tests passed!


In [107]:
def temperatures_average(temps: jnp.array) -> jnp.array:
    assert len(temps) % 7 == 0
    return einops.reduce(temps, "(h 7) -> h", "mean")

temps = jnp.array([71, 72, 70, 75, 71, 72, 70, 75, 80, 85, 80, 78, 72, 83], dtype=jnp.float32)
expected = [71.571, 79.0]
assert_all_close(temperatures_average(temps), jnp.array(expected))

Tests passed!


In [112]:
def temperatures_differences(temps: jnp.array) -> jnp.array:
    assert len(temps) % 7 == 0
    return temps - einops.repeat(temperatures_average(temps), "h -> (h w)", w=7)

expected = [-0.571, 0.429, -1.571, 3.429, -0.571, 0.429, -1.571, -4.0, 1.0, 6.0, 1.0, -1.0, -7.0, 4.0]
actual = temperatures_differences(temps)
assert_all_close(actual, jnp.array(expected))

Tests passed!


In [114]:
def temperatures_normalized(temps: jnp.array) -> jnp.array:
    avg = einops.reduce(temps, "(w 7) -> w", "mean")
    std = einops.reduce(temps, "(h 7) -> h", jnp.std)
    return (temps - einops.repeat(avg, "w -> (w 7)")) / einops.repeat(std, "w -> (w 7)")

expected = [-0.333, 0.249, -0.915, 1.995, -0.333, 0.249, -0.915, -0.894, 0.224, 1.342, 0.224, -0.224, -1.565, 0.894]
actual = temperatures_normalized(temps)
assert_all_close(actual, jnp.array(expected))

AssertionError: 
Not equal to tolerance rtol=0, atol=0.001

Mismatched elements: 14 / 14 (100%)
Max absolute difference among violations: 0.16026008
Max relative difference among violations: 0.08194579
 ACTUAL: array([-0.359214,  0.269405, -0.987833,  2.15526 , -0.359214,  0.269405,
       -0.987833, -0.966092,  0.241523,  1.449138,  0.241523, -0.241523,
       -1.690661,  0.966092], dtype=float32)
 DESIRED: array([-0.333,  0.249, -0.915,  1.995, -0.333,  0.249, -0.915, -0.894,
        0.224,  1.342,  0.224, -0.224, -1.565,  0.894], dtype=float32)

In [116]:
def normalize_rows(matrix: jnp.array) -> jnp.array:
    row_norms = matrix.norm(dim=1, keepdim=True)  # keepdim=True
    return matrix / row_norms

matrix = jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
expected = jnp.array([[0.267, 0.535, 0.802], [0.456, 0.570, 0.684], [0.503, 0.574, 0.646]])
assert_all_close(normalize_rows(matrix), expected)

AttributeError: 'jaxlib.xla_extension.ArrayImpl' object has no attribute 'norm'